# Model subclassing and custom training loops

> In the section, we're going to look at ways to build fully customizable deep learning models and layers, as well as custom training loops. This is the summary of lecture "Customizing your model with Tensorflow 2" from Coursera.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, Coursera, Deep_Learning, Tensorflow]
- image: 